In [ ]:
from __future__ import print_function
from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity='all'

from matplotlib import pyplot as plt
plt.style.use(['science','no-latex','grid','high-contrast','notebook']) # high-contrast指的是配色的选取, 去除notebook以后，在notebook里面显示得不方便
%pylab inline

# %load_ext autoreload
# %autoreload 2
import numpy as np
import pandas as pd 
pd.options.display.max_rows=5  # 设置dataframe最多的显示行数
import os
import sys
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())
    print(f"After inserting, sys.path is {sys.path}")

# more_itertools

## chunked

In [5]:
from more_itertools import chunked
to_process= range(1,20) 

after_process= [ sum(x) for x in chunked(to_process, 4)]
print(after_process)


[10, 26, 42, 58, 54]


# numpy常用/好用API

## np.tile
针对整个数组进行操作

In [81]:
mat = np.array([[1,2], [3, 4]])

**只能通过np.tile的方式调用**

**默认横线tile**

In [84]:
np.tile(mat,(1, 4))
np.tile(mat, 4)

array([[1, 2, 1, 2, 1, 2, 1, 2],
       [3, 4, 3, 4, 3, 4, 3, 4]])

array([[1, 2, 1, 2, 1, 2, 1, 2],
       [3, 4, 3, 4, 3, 4, 3, 4]])

In [83]:
np.tile(mat, (3, 1))

array([[1, 2],
       [3, 4],
       [1, 2],
       [3, 4],
       [1, 2],
       [3, 4]])

## np.repeat 逐元素进行操作

In [78]:
np.repeat(7.,4)

array([7., 7., 7., 7.])

**通过np调用 以及 array的API调用，是一样的效果**

In [79]:
a=np.array([10,20])
a.repeat([3,2])

np.repeat(a,[3,2])

array([10, 10, 10, 20, 20])

array([10, 10, 10, 20, 20])

**同样，也不会改变shape**

In [80]:
a=np.array([[10,20],[30,40]])
a.repeat([3,2],axis=0)

a.repeat([3,2],axis=1)

array([[10, 20],
       [10, 20],
       [10, 20],
       [30, 40],
       [30, 40]])

array([[10, 10, 10, 20, 20],
       [30, 30, 30, 40, 40]])

# pandas常用API

先构造一个数据

In [22]:
data = [[0,1,2] for _ in range(2)]
data.append([1,1,2])
df = pd.DataFrame(data,columns =['a','b','c']) 
df

,a,b,c
0,0,1,2
1,0,1,2
2,1,1,2


##  查看内存占用

In [4]:
df.memory_usage(deep=True).sum() / 1024**2  # memory usage (MB)
df.info(memory_usage='deep')

0.00019073486328125

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       3 non-null      int64
 1   1       3 non-null      int64
 2   2       3 non-null      int64
dtypes: int64(3)
memory usage: 200.0 bytes


## 灵活操作(利用pandas的优化API)

In [30]:
df.a.isin([0,1])
df[df.a.isin([0,1])]  # select via bool matrix
df.select_dtypes(["int16"])  # select via dtype
df.select_dtypes(["int"])
df.a.unique()

0    True
1    True
2    True
Name: a, dtype: bool

,a,b,b
0,0,1,2
1,0,1,2
2,1,1,2


""
0
1
2


,a,b,b
0,0,1,2
1,0,1,2
2,1,1,2


array([0, 1])

In [20]:
df.columns
df.drop_duplicates((["b","a"]))
df.drop_duplicates((["b"]))

Index(['a', 'b', 'b'], dtype='object')

,a,b,b
0,0,1,2
2,1,1,2


,a,b,b
0,0,1,2


## groupby常见使用方法

一般是喜欢和apply,filter等一起操作 , 输出的size会跟聚合的情况有关

In [38]:
df

,a,b,c,rank_test,rank_test2
0,0,1,2,1.5,1
1,0,1,2,1.5,1
2,1,1,2,1.0,1


In [39]:
df.groupby(["a","b"]).apply(sum)
df.groupby(["a","b"])["c"].apply(sum)

,,a,b,c,rank_test,rank_test2
a,b,,,,,
0,1,0.0,2.0,4.0,3.0,2.0
1,1,1.0,1.0,2.0,1.0,1.0


a  b
0  1    4
1  1    2
Name: c, dtype: int64

tranform则不会改变size （[refer](https://www.bilibili.com/video/BV1kJ411V7pb?from=search&seid=5227670558409086690)）

In [40]:
df.groupby(["a","b"]).transform(sum) 

,c,rank_test,rank_test2
0,4,3.0,2
1,4,3.0,2
2,2,1.0,1


其他用法

In [ ]:
df.groupby(["a","b"]).rank() # return a DataFrame type
df.groupby(["a"])["b"].rank() 
    # 默认的排序方式是average-->针对重合的如何排名
df['rank_test'] = df.groupby(["a","b"]).rank()
    # 其实这个astype(int)完全可以通过groupby的method参数进行调整
df['rank_test2'] = df.groupby(["a"])["b"].rank().astype(int)
df

In [22]:
np.random.choice(a=df.a, size=3, replace=False, p=[0.2, 0.1, 0.7])

array([1, 0, 0])

## python list , numpy , pandas 视图与副本

In [55]:
import numpy as np
import pandas as pd

listData=[ [1,2,3],[1,2,3],[1,2,3] ]
npData = np.array(listData)
pdData = pd.DataFrame(listData,columns = ['a','b','c'],index=['d','e','f'])

<div class="alert alert-block alert-info">
<b>list</b> 只有切片会返回副本（但还是浅拷贝），其余操作都是视图 , 除非索引到了
</div>

In [49]:
listData_slice = listData[:]
listData_slice[2]=1
listData_slice
listData

[[1, 2, 3], [1, 2, 3], 1]

[[1, 2, 3], [1, 2, 3], [1, 2, 3]]

In [56]:
listData_slice = listData[2]
listData_slice[2]=9
listData_slice
listData

[1, 2, 9]

[[1, 2, 3], [1, 2, 3], [1, 2, 9]]

<div class="alert alert-block alert-info">
<b>np.array</b> 所有都是返回视图，底层的所有数据都是公用的
</div>

In [46]:
npData_slice = npData
npData_slice[2] = 9    # 哈哈，又掌握了一个 给array整行赋值的一个方式
npData_slice
npData

array([[1, 2, 3],
       [1, 2, 3],
       [9, 9, 9]])

array([[1, 2, 3],
       [1, 2, 3],
       [9, 9, 9]])

<div class="alert alert-block alert-info">
<b>DataFrame</b> 是基于np.array的一个数据结构，自然也是如此
</div>

In [47]:
pdData_slice = pdData
pdData_slice[2] = 9
pdDddata_slice
pdData

,a,b,c,2
d,1,2,3,9
e,1,2,3,9
f,1,2,3,9


,a,b,c,2
d,1,2,3,9
e,1,2,3,9
f,1,2,3,9


## read_csv 和 to_csv注意点

In [ ]:
pdData.to_csv( filePath, data_to_store , header=None,index=False ) # 默认index和header都会被记录下来（然后默认读的时候没有，hh搞笑）
tmp_data = pdData.read_csv(filePath,header=None)  # 默认是不读入index的，如果有且要读入的话，要设置index_col=0参数

# argparse
[官网的reference](https://docs.python.org/zh-cn/3/library/argparse.html#argparse.ArgumentParser.add_argument)

**sample**
* 没有指定dest,则默认为第一个长选项，没有再找短选项；指定了dest，就赋值到dest变量上

* 测试方法：parser.parse_args('-f 1 -x 2'.split())

```bash
>>> parser = argparse.ArgumentParser()
>>> parser.add_argument('-f', '--foo-bar', '--foo') 
>>> parser.add_argument('-x', '-y')
>>> parser.parse_args('-f 1 -x 2'.split())
Namespace(foo_bar='1', x='2')
>>> parser.parse_args('--foo 1 -y 2'.split())
Namespace(foo_bar='1', x='2')
```

* 一般来说长参数选项(--xx)的名字会比较长，但是短参数选项(-x)的名字会比较短。但是我自己并感到无所谓。

## 整体流程

In [4]:
import argparse
parser = argparse.ArgumentParser(description="Demo of argparse")
parser.add_argument( '--xx','-x' ,dest="hesyTestVar",help="input a float to test",type=float )
args = parser.parse_args()

## 参数解释

### type

In [4]:
parser.add_argument('-name', type=str, required=True)  # required mandatorily ; type:int/bool

### metavar & nargs

In [4]:
parser.add_argument('--integers', metavar='3 4 5', nargs='+', help='an integer for the accumulator')

usage: lp_decision.py [-h] [--integers 3 4 5 [3 4 5 ...]] [--sum]  # nargs='+'表示会把所有的输入集合到一个list里面

In [4]:
parser.add_argument('--integers', metavar='3 4 5', help='an integer for the accumulator')

usage: lp_decision.py [-h] [--integers 3 4 5] [--sum]    

### action

In [ ]:
# store_true 像个开关选项一样
parser.add_argument('-t',action='store_true')

# store_const 是否存储到const常量中去
parser.add_argument('--sum', dest='accumulate', action='store_const',const=sum, default=max,
                    help='sum the integers (default: find the max)')

<br><br>

----